In [1]:
import requests
from io import BytesIO
import PIL.Image
from IPython.display import display,clear_output,HTML
from IPython.display import Image as DisplayImage
import base64
import json
from io import StringIO
import ipywidgets as widgets
import sys
import time
import imageio
import numpy as np
import io
import helper 
import datetime
import os
from stable_baselines3 import PPO, DQN
import pandas as pd
import torch as th
from stable_baselines.bench import Monitor

In [2]:
###############################################################################
# Training params
###############################################################################
training_steps_exponent = 2 #20 # Set to ~ 1mil
epoch_training_steps = 2**training_steps_exponent
epochs = 100
###############################################################################
# Evaluation params
###############################################################################
n_eval_episodes = 1000
###############################################################################
driving_agent = 'pelican'
config_fpath = '/data/configs/RBBRs_against_rulebased/monday_26x33_cl/monday_26x33_Panther_Agent_Move_North_pelican.json'

In [3]:
# Example where UIOutput is switched off. If ui_on is set to true, then UIOutput
# is generated each iteration. This is rather costly. By setting ui_on to false
# we get ~10x speed up... 
env = helper.get_env(driving_agent, config_fpath,
                    random_panther_start_position = True,
                    random_pelican_start_position = True,
                    max_illegal_moves_per_turn = 2, #self.max_illegal_moves_per_turn,
                    sparse = False,
                    curriculum_learning = True,
                    normalise = True,
                    ui_on=False) #!!! To speed-up set ui_on to false 

env = Monitor(env, './logs')

policy_kwargs = dict(activation_fn=th.nn.ReLU, net_arch=[128, 64])

model = DQN('MlpPolicy',
            env, 
            policy_kwargs=policy_kwargs,                
            buffer_size=1000000,
            learning_starts=200,
            batch_size=128,
            gamma=0.95,
            #tau=0.0, # Soft target update uses: target_param.data.mul_(1 - tau) + ...
            target_update_interval=1000, # So every 10 - 20 Episodes
            exploration_fraction=.3, #exploration_fraction: fraction of entire training period over which the exploration rate is reduced
            exploration_initial_eps=1.0, # exploration_initial_eps: initial value of random action probability
            exploration_final_eps=0.1, # exploration_final_eps: final value of random action probability
            verbose=1)

model.learn(epoch_training_steps)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [5]:
# make a video
print("****** STARTING TO MAKE VID *******")

# To generate a video we need an env with UIOutput enabled:
env_video = helper.get_env(driving_agent, config_fpath,
                            random_panther_start_position = True,
                            random_pelican_start_position = True,
                            max_illegal_moves_per_turn = 2, #self.max_illegal_moves_per_turn,
                            sparse = False,
                            curriculum_learning = True,
                            normalise = True,
                            ui_on=True) # Switch UIOutput back on for video generation 

video_file_path = os.path.join('example_video.mp4')
basewidth,hsize = helper.make_video(model, env_video, video_file_path, n_steps = 200,
                                                      basewidth=1024, 
                                                      fps=2)
print("****** FINISHED MAKING VID *******")


****** STARTING TO MAKE VID *******


INFO:gym_plark.envs.plark_env_cl:Len outcome tracker: 1, win percentage 0.000000


****** FINISHED MAKING VID *******


In [6]:
video = io.open(video_file_path, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" width="'''+str(basewidth)+'''" height="'''+str(hsize)+'''" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))